# One use notebook

In [1]:
import pandas as pd

In [2]:
def add_suff(in_str, suffix):
    return in_str + " " + suffix if in_str != "База" else in_str

In [3]:
infile = "../nts_source/input.xlsx"
outfile = "../tests/data/test05_nts_simple.xlsx"

## Deal with cities

In [24]:
df_cities = pd.read_excel(infile, sheet_name="Cities", skiprows=3)
df_cities.head()

,ID,Name
0,1,База
1,2,Район 1
2,3,Район 2
3,4,Район 3
4,5,Район 4


In [25]:
df_cities = df_cities.drop("ID", axis=1).rename(columns={"Name":"City"})

In [26]:
df1 = df_cities.copy()
df2 = df_cities.copy()
df3 = df_cities.copy()

df1["CityNew"] = df1.City.apply(lambda r: add_suff(r, "w1"))
df2["CityNew"] = df2.City.apply(lambda r: add_suff(r, "w2"))
df3["CityNew"] = df3.City.apply(lambda r: add_suff(r, "w3"))

df_cities = pd.concat([df1, df2, df3], axis=0).drop_duplicates()
df1 = df2 = df3 = None

In [27]:
df_cities

,City,CityNew
0,База,База
1,Район 1,Район 1 w1
2,Район 2,Район 2 w1
3,Район 3,Район 3 w1
4,Район 4,Район 4 w1
5,Район 5,Район 5 w1
1,Район 1,Район 1 w2
2,Район 2,Район 2 w2
3,Район 3,Район 3 w2
4,Район 4,Район 4 w2


In [8]:
#df_cities.drop("City", axis=1).rename(columns={"CityNew":"City"}).to_excel(infile, sheet_name="Cities", index=False)

In [28]:
df_cities_product = pd.DataFrame(index=pd.MultiIndex.from_product([df_cities.CityNew.values, 
                                                    df_cities.CityNew.values], names=["From", "To"])).reset_index()

In [29]:
df_cities_product = df_cities_product[df_cities_product.From != df_cities_product.To]

In [30]:
df_cities_product = df_cities_product.merge(df_cities, 
                                            left_on="From", 
                                            right_on="CityNew").drop("CityNew", 
                                                                     axis=1).rename(columns={"City": "FromOld"})

In [31]:
df_cities_product = df_cities_product.merge(df_cities, 
                                            left_on="To", 
                                            right_on="CityNew").drop("CityNew", 
                                                                     axis=1).rename(columns={"City": "ToOld"})

In [33]:
#df_cities_product

## Deal with routes

In [41]:
df_routes = pd.read_excel(infile, sheet_name="Routes", skiprows=3)
df_routes.head()

,Route_Id,from_Name,from_ID,to_Name,to_ID,Time,Distance
0,1,База,1,Район 1,2,2,100
1,2,Район 2,3,Район 1,2,2,100
2,3,Район 3,4,Район 1,2,4,200
3,4,Район 4,5,Район 1,2,5,300
4,5,Район 5,6,Район 1,2,5,300


In [42]:
df_routes = df_routes.drop(["Route_Id", "from_ID", "to_ID"], axis=1).rename(columns={"from_Name":"FromOld", "to_Name":"ToOld"})

In [43]:
df_routes = df_cities_product.merge(df_routes, 
                                    how="left", 
                                    on=["FromOld", "ToOld"]).drop(["FromOld", "ToOld"], axis=1).fillna(0)
#df_routes

In [45]:
df_routes.head(20)

,From,To,Time,Distance
0,База,Район 1 w1,2.0,100.0
1,Район 2 w1,Район 1 w1,2.0,100.0
2,Район 3 w1,Район 1 w1,4.0,200.0
3,Район 4 w1,Район 1 w1,5.0,300.0
4,Район 5 w1,Район 1 w1,5.0,300.0
5,Район 1 w2,Район 1 w1,0.0,0.0
6,Район 2 w2,Район 1 w1,2.0,100.0
7,Район 3 w2,Район 1 w1,4.0,200.0
8,Район 4 w2,Район 1 w1,5.0,300.0
9,Район 5 w2,Район 1 w1,5.0,300.0


## Deal with orders

In [46]:
df_orders = pd.read_excel(infile, sheet_name="OrdersBase", skiprows=3)
df_orders.head()

,ID,Customer,City,city_ID,Quantity,Price_per_unit,Deliver_From,Deliver_To
0,1,Покупатель 1,Район 1,2,5000,100,1,8
1,2,Покупатель 2,Район 1,2,5046,100,1,8
2,3,Покупатель 3,Район 2,3,642,100,1,8
3,4,Покупатель 4,Район 2,3,4865,100,1,8
4,5,Покупатель 5,Район 3,4,218,100,1,8


In [52]:
grp_cols = ["City", "Price_per_unit", "Deliver_From", "Deliver_To"]
rename_cols = {"Deliver_From": "Deliver_from" , "Deliver_To": "Deliver_to", "Quantity": "Demand"}
df_orders = df_orders.groupby(cols)["Quantity"].sum().to_frame().reset_index().rename(columns=rename_cols)

In [56]:
df_orders["RN"] = df_orders.sort_values("Deliver_from").groupby("City").cumcount()

In [66]:
df_orders["City"] = df_orders["City"] + " w" + (df_orders["RN"] + 1).astype("str")

In [68]:
df_orders = df_orders.drop("RN", axis=1)

## Deal with  cars

In [69]:
df_cars = pd.read_excel(infile, sheet_name="Cars", skiprows=3)
df_cars.head()

,car_ID,Capacity_Max,Price_per_km
0,1,20000,200
1,2,20000,200
2,3,20000,200
3,4,7500,200
4,5,15000,300


In [71]:
df_cars = df_cars.rename(columns={"car_ID": "CarName", "Capacity_Max": "Capacity"})

## Deal with  parameters

In [77]:
df_param =  pd.read_excel(infile, sheet_name="Parametres", skiprows=1)
df_param

,Unnamed: 0,Unnamed: 1
0,minPeriod,0
1,maxPeriod,25


In [78]:
df_param.columns = ["ParamName", "ParamValue"]

## Save to file

In [81]:
writer = pd.ExcelWriter(outfile, engine="xlsxwriter")

df_param.to_excel(writer, sheet_name="Parameters", index=False)
df_cities.drop("City", axis=1).rename(columns={"CityNew":"City"}).to_excel(writer, sheet_name="Cities", index=False)
df_cars.to_excel(writer, sheet_name="Vehicles", index=False)
df_orders.to_excel(writer, sheet_name="Orders", index=False)
df_routes.to_excel(writer, sheet_name="Routes", index=False)
writer.save()
writer.close()